res# 50 оттенков градиентного спуска 

В этом задании вам предстоит реализовать линейный классификатор и натренировать его, используя различные модификации градинетного спуска. Тетрадка позаимствована с [шадовского курса по нейронкам.](https://github.com/yandexdataschool/Practical_DL/blob/master/week01_backprop/adapdive_sgd/adaptive_sgd.ipynb)

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import time
from scipy.optimize import minimize
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [2]:
def expand(X):
    """
 
    """
    ones=np.ones(len(X))
    ones.shape=(len(X),1)
    m=X[:,0]*X[:,1]
#     print(m.values)
    m.shape=(len(X),1)
    X=np.hstack((X,ones))
    return(X)

In [3]:
df=pd.read_excel("Устойчивост эл_сети.xlsx",engine='openpyxl',sheet_name='Data_for_UCI_named')
X=df[df.columns[0:12]].values
for i in range(X.shape[1]):
    X[:,i]=(X[:,i]-X[:,i].min())/(X[:,i].max()-X[:,i].min())
X=expand(X)
y=(df.stabf=='stable')*1
y=y.values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)


In [4]:
cfl=DecisionTreeClassifier(max_depth=5)
cfl.fit(X_train,y_train)



test_score=2*roc_auc_score(y_test,cfl.predict_proba(X_test)[:,1])-1
train_score=2*roc_auc_score(y_train,cfl.predict_proba(X_train)[:,1])-1
print(test_score,train_score)

0.6800876303116883 0.7713121866398394


In [5]:
def probability(X, w):
    """
    Принимает на вход матрицу фичей и вектор весов
    Возвращает предсказание вероятность того, что y = 1 при фиксированных x, P(y=1|x)
    
    :param X: расширенная матрица фичей [n_samples,6] (expanded)
    :param w: вектор весов [6]
    :returns: вектор вероятностей
    """
    linear=np.dot(X,w)
    
    #linear[linear<0]=linear[linear<0]/10000
    #linear[linear>1]=linear[linear>1]/10000
    
    linear[linear<0]=0
    linear[linear>1]=1
    #linear[linear>1]=1/(1+np.exp(-linear[linear>1]))*2-1
    
    
    return linear


In [6]:
def compute_loss(X, y, w, L):
    """
    Принимает на вход матрицу весов, вектор ответов и вектор весов.
    Выдаёт на выход значение функции потерь, расчитанное по формуле выше.
    """
    #L=1000
        
    p1=probability(X,w)
    loss=np.sum((L-(L+1)*y)*p1)
   
    return loss
    # ваш код здесь

In [7]:
def new(w):
    return compute_loss(X_train,y_train,w,L)

def секущая_гипер_плоскость():
    c=0.1
    A=[]
    sdt=time.time()
    for i in range(100):
        vBeta0= np.array([(np.random.rand(X_train.shape[1])-0.5)*c])
        res=minimize(new, x0=vBeta0,method='Nelder-Mead',options={'disp': False,'adaptive':True})
        A+=[[i,res.fun,res.x]]
#         print(A[i])
    A=pd.DataFrame(A)
    A=A.sort_values(1).head(1)
    коэфициенты_гипер_плоскости=A[2].values[0]
    print(time.time()-sdt,A[2].values[0])
    
    return A[2].values[0]
# 

def секатор(X, w):
    linear=np.dot(X,w)
    linear[linear<0]=0
    return np.sign(linear)

In [8]:
L=100
секущая_гипер_плоскость()

7.548850774765015 [  -90.39835935  -130.72735036    43.3554188  -1314.72861029
  -273.16856293  -651.92829009  1518.14086062   150.9322085
 -1280.05759934 -3450.07295705   -99.56966386   212.24082809
  -191.19456037]


array([  -90.39835935,  -130.72735036,    43.3554188 , -1314.72861029,
        -273.16856293,  -651.92829009,  1518.14086062,   150.9322085 ,
       -1280.05759934, -3450.07295705,   -99.56966386,   212.24082809,
        -191.19456037])

In [8]:
def секущая_гипер_плоскость2():
    c=0.1
    A=[]
    sdt=time.time()
    for i in range(1000):
        vBeta0= np.array((np.random.rand(X_test.shape[1])-0.5)*c)
        start_loss=compute_loss(X_exp, y, vBeta0, L)
        res=minimize(new, x0=vBeta0,method='Nelder-Mead',options={'disp': False,'adaptive':True,'maxiter':100,'xatol':0.3})
        A+=[[i,res.fun,res.x,start_loss,vBeta0]]
    A=pd.DataFrame(A)
    
    return A

In [9]:


def секущая_гипер_плоскость3():
    c=0.1
    A=[]
    B=[]
    sdt=time.time()
    for i in range(100):
        vBeta0= np.array((np.random.rand(X_test.shape[1])-0.5)*c)
        res=minimize(new, x0=vBeta0,method='Nelder-Mead',options={'disp': False,'adaptive':True,'maxiter':10,'xatol':1})
        
        
        if res.fun<0:       
            A+=[res.x]
    
    
    for vBeta0 in A:
        res=minimize(new, x0=vBeta0,method='Nelder-Mead',options={'disp': False,'adaptive':True,'xatol':1})
        
        for k in range(2):
            res=minimize(new, x0=res.x,method='Nelder-Mead',options={'disp': False,'adaptive':True})
        
        B+=[[res.fun,res.x,vBeta0]]
            
            
    B=pd.DataFrame(B)
   
    
    return B

In [12]:

sdt=time.time()

L=100
df=секущая_гипер_плоскость3()

print(time.time()-sdt)

6.137999057769775


In [13]:
df

,0,1,2
0,-419.190386,"[-4.604848595729167, 0.19389750231405056, -2.7...","[-0.02123255683576636, 0.01242878403912803, -0..."
1,-279.059812,"[1.4613195796460516, -18.851873403580008, 0.65...","[0.03396180671749107, -0.0399700220127847, -0...."
2,-201.000000,"[-46.85205617961829, -260.91730619478653, -96....","[0.006181608291660418, -0.02119352188091247, 0..."
3,-457.717135,"[-1.7481468560774447, -1.6835375039727585, -9....","[-0.033379350444954825, -0.009900942025260838,..."


In [17]:

A=[]
for i in df[df[0]<0][1]:
    
    vBeta0= i
    start_loss=compute_loss(X, y, vBeta0, L)
    res=minimize(new, x0=vBeta0,method='Nelder-Mead',options={'disp': False,'adaptive':True})
    A+=[[i,res.fun,res.x,start_loss,vBeta0]]
A=pd.DataFrame(A)
A=A.sort_values(by=1)[0][0]
    
    

In [21]:
A

array([-4.6048486 ,  0.1938975 , -2.75157864, -5.77063014, -0.02458975,
       -0.10405557,  0.30596697, -0.06598948, -2.72577222, -4.92789997,
       -3.58344777, -2.42201091,  8.92644437])

In [22]:
cut=секатор(X_train, A)
# X_train=X_train[cut==0]
# y_train=y_train[cut==0]



In [26]:
X_train[[True, False]]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 7000 but corresponding boolean dimension is 2

In [14]:
df=pd.read_excel("Устойчивост эл_сети.xlsx",engine='openpyxl',sheet_name='Data_for_UCI_named')
df['y']=y
df['prob']=probability(X, A)
df.to_clipboard()

In [32]:
a = np.array([[1, 2], [3, 4]])
a[0]

array([1, 2])

In [15]:
Test=pd.DataFrame(y_test)

In [16]:
Test['Predict']=probability(X_test, A)

In [17]:
Test.to_clipboard()

In [44]:
abc = np.zeros(len(y_test))

In [46]:
abc

array([0., 0., 0., ..., 0., 0., 0.])

In [48]:
a = np.array([0, 1, 0, 0, 1])
train_preds = pd.DataFrame(y, columns=['TARGET'])
train_preds['PEDICTIONS'] = 0
train_preds

,TARGET,PEDICTIONS
0,0,0
1,1,0
2,0,0
3,0,0
4,0,0
...,...,...
9995,0,0
9996,1,0
9997,1,0
9998,0,0


In [49]:
bools = [True, True, True] + [False] * 9997 

In [50]:
len(bools)

10000

In [52]:
train_preds.PEDICTIONS[bools] = 0.5

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
train_preds

,TARGET,PEDICTIONS
0,0,0.5
1,1,0.5
2,0,0.5
3,0,0.0
4,0,0.0
...,...,...
9995,0,0.0
9996,1,0.0
9997,1,0.0
9998,0,0.0
